<center><img src = "covid%2019%20website%20banner.jpg" align="center"/></center>
# Covid-19
>"CDC is responding to a pandemic of respiratory disease spreading from person-to-person caused by a novel (new) coronavirus. The disease has been named “coronavirus disease 2019” (abbreviated “COVID-19”). This situation poses a serious public health risk. The federal government is working closely with state, local, tribal, and territorial partners, as well as public health partners, to respond to this situation. COVID-19 can cause mild to severe illness; most severe illness occurs in older adults." - Centers for Disease Control and Prevention

In [1]:
from __future__ import print_function
#import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import folium

In [2]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [3]:
# Load the shape of the zone (US states)
# Find the original file here: https://github.com/python-visualization/folium/tree/master/examples/data
# You have to download this file and set the directory where you saved it
# state_geo = os.path.join('/Users/y.holtz/Desktop/', 'us-states.json')
##world_geo = os.path.join('/Users/Nolan Khounborin/voila-heroku/notebooks','world-countries.json')
##m = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom=6, min_zoom=2)
 
# Load the unemployment value of each state
# Find the original file here: https://github.com/python-visualization/folium/tree/master/examples/data
# state_unemployment = os.path.join('/Users/y.holtz/Desktop/', 'US_Unemployment_Oct2012.csv')
# state_data = pd.read_csv(state_unemployment)
 
# Initialize the map:
# world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom=6, min_zoom=2)
 
# Add the color for the chloropleth:
##m.choropleth(
 ##geo_data=world_geo,
 ##name='choropleth',
 ##data=country_df,
 ##columns=['Country_Region', 'Deaths'],
 ##key_on='feature.properties.name',
 ##fill_color='RdPu',
 ##fill_opacity=0.7,
 ##line_opacity=0.2,
 ##legend_name='# Of Deaths',
 ##bins=[0,1000,5000, 15000, 20000, 25000, 30000,45000]
##)
##folium.LayerControl().add_to(m)
 
# Save to html
##m.save('#292_folium_chloropleth_USA1.html')
##m
 

In [4]:
country_df.columns = map(str.lower, country_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
death_df.columns = map(str.lower, death_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)

In [5]:
confirmed_df = confirmed_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
recovered_df = recovered_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})

In [6]:
sorted_country_df = country_df.sort_values('confirmed', ascending=False)

#Percentage of the world total deaths of each country
>The information used is from the previous day

In [16]:
fig = px.pie(sorted_country_df, names='country_region', values='deaths', title='Percentage of Total Deaths', width=700, height=700)
fig.show();

In [8]:
#sorted_country_df

In [9]:
fig = go.FigureWidget(layout = go.Layout())
def highlight_col(x):
    r = 'background-color: red'
    p = 'background-color: purple'
    g = 'background-color: grey'
    
    temp_df = pd.DataFrame('', index=x.index, columns = x.columns)
    temp_df.iloc[:,4] = p
    temp_df.iloc[:,5] = r
    temp_df.iloc[:,6] = g
    return temp_df

In [10]:
#sorted_country_df.style.apply(highlight_col, axis = None)

#Check the worst hit countries
>Slide the slider to add or subtract countries (max 30)

In [11]:
def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x='country_region', y='confirmed', size='confirmed',
                 color='country_region', hover_name='country_region', size_max=60)
    fig.update_layout(
    title=str(n) + " Worst hit countries",
    xaxis_title = "Countries",
    yaxis_title = "Confirmed Cases",
    width = 700)
    fig.show()
    
interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [12]:
def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

#Check the Confirmed and Deaths of any country or the World
>Enter the name of the country with capitalization (United States is US, ex: Italy) or World

In [13]:
interact(plot_cases_of_a_country, country='World')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

#Global Death Rate of each country
>Hover over a circle to see the details of a country

In [14]:
world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom=6, min_zoom=2)

In [15]:
for i in range(len(confirmed_df)):
    folium.Circle(
    location =[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
    fill = True,
    radius = (int((np.log(confirmed_df.iloc[i,-1]+1.00001))) + 0.2)*50000,
    fill_color = 'blue',
    color = 'red',
    tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>" + confirmed_df.iloc[i]['country'] + "</h4>" 
                    "<hr style='margin:10px;color: white;'>" +
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: " +str(confirmed_df.iloc[i,-1])+"</li>" +
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>" +
                    "</ul></div>",
    ).add_to(world_map)
        
world_map

>Covid-19 Banner from https://www.med.navy.mil/sites/nhcl/News/Pages/COVID-19.aspx
<br>
>Covid-19 information from John Hopkins University